In [2]:
import pandas as pd
import numpy as np
import shutil as sht
import os
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.models  import load_model
from tensorflow.keras  import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import binary_crossentropy, categorical_crossentropy
from tensorflow.keras import callbacks



2022-08-30 17:43:48.004444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 17:43:48.004464: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# source_folder = r"/home/lepton/Desktop//"
# destination_folder = r"/home/lepton/code/vivekptl9/Anaemia_Classification//"

# # fetch all files
# for file_name in os.listdir(source_folder):
#     # construct full file path
#     source = source_folder + file_name
#     destination = destination_folder + file_name
#     # copy only files
#     if os.path.isfile(source):
#         sht.copy(source, destination)
#         print('copied', file_name)

In [4]:
with open('/home/lepton/Downloads/healthy.npy', 'rb') as f:
    healthy = np.load(f)
with open('/home/lepton/Downloads/sick.npy', 'rb') as f:
    sick = np.load(f)

In [5]:
n_samples = 500

X_healthy = healthy[:n_samples]
X_sick = sick[:n_samples]
X_train = tf.convert_to_tensor(np.concatenate((X_healthy,X_sick))) #sampl X_healthy and X_sick are numpy arry

y_train = []
for i in range(0, n_samples):
    y_train.append(0)

for i in range(0, n_samples):
    y_train.append(1)

y_train_healthy = 0

y_train = tf.convert_to_tensor(y_train)

2022-08-30 17:43:59.154869: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-30 17:43:59.155138: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vivek-lepton): /proc/driver/nvidia/version does not exist
2022-08-30 17:43:59.156683: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
n_test = int(n_samples*0.2)
X_test_healthy = healthy[n_samples:(n_samples + n_test)]
X_test_sick = sick[n_samples:(n_samples + n_test)]
X_test = tf.convert_to_tensor(np.concatenate((X_test_healthy, X_test_sick)))
X_test.shape
y_test = []

for i in range(0, n_test):
    y_test.append(0)

for i in range(0, n_test):
    y_test.append(1)
y_test = tf.convert_to_tensor(y_test)


In [7]:
model = models.Sequential()

model.add(layers.Conv2D(30, kernel_size=(3, 3), activation='relu', padding = 'same',input_shape=(224, 224, 3)))
#model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(20, kernel_size=(3), activation='relu')) # kernel_size = 3 <==> (3, 3)
#model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(10, kernel_size=(3), activation='relu')) # kernel_size = 3 <==> (3, 3)
#model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Conv2D(5, kernel_size=(3), activation='relu')) # kernel_size = 3 <==> (3, 3)
#model.add(layers.MaxPool2D(pool_size=(2,2))) 

model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid')) 

model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 30)      840       
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 20)      5420      
                                                                 
 conv2d_2 (Conv2D)           (None, 220, 220, 10)      1810      
                                                                 
 conv2d_3 (Conv2D)           (None, 218, 218, 5)       455       
                                                                 
 flatten (Flatten)           (None, 237620)            0         
                                                                 
 dense (Dense)               (None, 1)                 237621    
                                                                 
Total params: 246,146
Trainable params: 246,146
Non-trai

In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[tf.keras.metrics.BinaryAccuracy()])
es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train, y_train,
          epochs=20,
          validation_split= 0.2,
          shuffle= True,
          callbacks = [es],  # Use early stopping in practice
          batch_size=10, 
          verbose=1)

Epoch 1/20
80/80 [==============================] - 142s 2s/step - loss: 0.8308 - binary_accuracy: 0.6050 - val_loss: 0.7131 - val_binary_accuracy: 0.0000e+00
Epoch 2/20
26/80 [========>.....................] - ETA: 1:33 - loss: 0.6867 - binary_accuracy: 0.6423

KeyboardInterrupt: 